In [ ]:
import numpy as np 
import string 
import sys 
from nltk.tokenize import RegexpTokenizer 
from nltk.corpus import stopwords 
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.utils import np_utils 
from keras.callbacks import ModelCheckpoint

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
stop=set(stopwords.words('english'))
punc=list(string.punctuation)
stop.update(punc)

In [ ]:
from google.colab import files
text = files.upload()

In [ ]:

file_name = "frank.txt"
text = text[file_name].decode("utf-8")

In [ ]:
def tokenize_words(text):
  text = text.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(text)
  filtered = filter(lambda token: token not in stop,tokens)
  return " ".join(filtered)
  
processed_input = tokenize_words(text)

In [ ]:
len(processed_input)

In [ ]:
char = sorted(list(set(processed_input)))
char_to_num = dict((c,i) for i,c in enumerate(char))

In [ ]:
input_len = len(processed_input)
vocab_len = len(char)
print("Total Number of Characters : ",input_len)
print("Total Vocab : ",vocab_len)

In [ ]:
seq_len = 100
x_data = []
y_data = []

In [ ]:
for i in range(0,input_len-seq_len,1):
  in_seq = processed_input[i:i + seq_len]
  out_seq = processed_input[i + seq_len]
  x_data.append([char_to_num[char] for char in in_seq])
  y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print("Patterns : ",n_patterns)

In [ ]:
x = np.reshape(x_data,(n_patterns,seq_len,1))
x = x/float(vocab_len)

In [ ]:
y = np_utils.to_categorical(y_data)

In [ ]:

model = Sequential()
model.add(LSTM(256,input_shape=(x.shape[1],x.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True ))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',optimizer='adam')

In [ ]:
# saving weights 
filepath = 'model_weights_saved.hdf5'
checkpoint = ModelCheckpoint(filepath,monitor='loss',verbose=1,save_best_only=True,mode='min')
desired_callbacks = [checkpoint]

In [ ]:
model.fit(x,y,epochs=1,batch_size=256,callbacks=desired_callbacks)